In [1]:
import pandas as pd

In [2]:
weather_data = pd.read_csv('impute/sorted_precipitation_data.csv', dtype={'week': 'str'})

print(weather_data.head())

  ESTACAO    week  precipitation_max  precipitation_avg
0    A601  201201               66.2          14.171429
1    A621  201201               11.2           2.714286
2    A624  201201               17.2           5.771429
3    A603  201201               19.0           9.685714
4    A604  201201               18.4           7.514286


In [3]:
stations_file_path = "impute/stations_RIO.csv"
output_file_path = "impute/sorted_precipitation_data_with_stations.csv"

stations_data = pd.read_csv(stations_file_path)

# Merge the data based on the station ID
merged_data = pd.merge(
    weather_data,
    stations_data[['id_station', 'lat', 'lon', 'lvl']],
    left_on='ESTACAO',
    right_on='id_station',
    how='left'
)

# Drop the now-redundant 'id_station' column
merged_data = merged_data.drop(columns=['id_station'])

# Save the merged data to a new CSV file
merged_data.to_csv(output_file_path, index=False)

print(f"Merged data saved to {output_file_path}")

Merged data saved to impute/sorted_precipitation_data_with_stations.csv


In [4]:
import pandas as pd
from pykrige.uk3d import UniversalKriging3D

def interpolate_precipitation_for_city(file_path, target_lat, target_lon, target_lvl):
    df = pd.read_csv(file_path)
    weeks = df['week'].unique()
    results = []

    for week in weeks:
        week_data = df[df['week'] == week]

        if len(week_data) < 3:
            continue

        x = week_data['lon'].values
        y = week_data['lat'].values
        z = week_data['lvl'].values

        p_max = week_data['precipitation_max'].values
        p_avg = week_data['precipitation_avg'].values

        uk_max = UniversalKriging3D(
            x, y, z, p_max,
            variogram_model='linear',
            drift_terms=['regional_linear']
        )
        interp_max, _ = uk_max.execute(
            'points', [target_lon], [target_lat], [target_lvl]
        )

        uk_avg = UniversalKriging3D(
            x, y, z, p_avg,
            variogram_model='linear',
            drift_terms=['regional_linear']
        )
        interp_avg, _ = uk_avg.execute(
            'points', [target_lon], [target_lat], [target_lvl]
        )

        results.append({
            'week': week,
            'precipitation_max': interp_max[0],
            'precipitation_avg': interp_avg[0]
        })

    return pd.DataFrame(results)


In [5]:
city_lat = -22.9
city_lon = -43.2
city_lvl = 20  # Example altitude

df_result = interpolate_precipitation_for_city(
    "impute/sorted_precipitation_data_with_stations.csv",
    city_lat, city_lon, city_lvl
)

print(df_result)


       week  precipitation_max  precipitation_avg
0    201201          16.709923           4.350179
1    201202          16.146143           3.404620
2    201203           3.609518           1.009952
3    201204           6.419282           1.541386
4    201205          10.671312           2.284658
..      ...                ...                ...
569  202248          18.829515           4.824378
570  202249           6.513012           1.365163
571  202250           8.935739           2.047114
572  202251          10.682593           3.356903
573  202252          10.316381           2.276391

[574 rows x 3 columns]
